In [ ]:
#| hide
#| default_exp export

# export
> Exporting a notebook to a library
- order: 4

In [ ]:
#| export
from nbdev.config import *
from nbdev.maker import *
from nbdev.imports import *
from nbdev.process import *

from fastcore.script import *
from fastcore.basics import *
from fastcore.imports import *
from fastcore.meta import *

from collections import defaultdict

In [ ]:
#| hide
from fastcore.test import *
from pdb import set_trace
from importlib import reload
from fastcore import shutil
from execnb.nbio import read_nb

In [ ]:
#| export
class ExportModuleProc:
    "A processor which exports code to a module"
    def begin(self): self.modules,self.in_all = defaultdict(L),defaultdict(L)
    def _default_exp_(self, cell, exp_to): self.default_exp = exp_to
    def _exporti_(self, cell, exp_to=None): self.modules[ifnone(exp_to, '#')].append(cell)
    def _export_(self, cell, exp_to=None):
        self._exporti_(cell, exp_to)
        self.in_all[ifnone(exp_to, '#')].append(cell)
    def __call__(self, cell):
        src = cell.source
        if not src: return
        if cell.cell_type=='markdown' and src.startswith('# '): self.modules['#'].append(cell)
    _exports_=_export_

Specify `dest` where the module(s) will be exported to, and optionally a class to use to create the module (`ModuleMaker`, by default).

Exported cells are stored in a `dict` called `modules`, where the keys are the modules exported to. Those without an explicit module are stored in the `'#'` key, which will be exported to `default_exp`.

In [ ]:
everything_fn = '../../tests/01_everything.ipynb'

exp = ExportModuleProc()
proc = NBProcessor(everything_fn, exp)
proc.process()
test_eq(exp.default_exp, 'everything')
assert 'print_function'  in exp.modules['#'][1].source
assert 'h_n' in exp.in_all['some.thing'][0].source

### `nb_export`

In [ ]:
#| export
def nb_export(nbname:str,        # Filename of notebook 
              lib_path:str=None, # Path to destination library.  If not in a nbdev project, defaults to current directory.
              procs=None,        # Processors to use
              name:str=None,     # Name of python script {name}.py to create.
              mod_maker=ModuleMaker,
              debug:bool=False,  # Debug mode
              solo_nb:bool=False # Export single notebook outside of an nbdev project.
             ):
    "Create module(s) from notebook"
    if lib_path is None: lib_path = get_config().lib_path if is_nbdev() else '.'
    exp = ExportModuleProc()
    nb = NBProcessor(nbname, [exp]+L(procs), debug=debug)
    nb.process()
    for mod,cells in exp.modules.items():
        if first(1 for o in cells if o.cell_type=='code'):
            all_cells = exp.in_all[mod]
            nm = ifnone(name, getattr(exp, 'default_exp', None) if mod=='#' else mod)
            if not nm:
                warn(f"Notebook '{nbname}' uses `#| export` without `#| default_exp` cell.\n"
                     "Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.\n"
                     "See https://nbdev.fast.ai/getting_started.html for more information.")
                return
            mm = mod_maker(dest=lib_path, name=nm, nb_path=nbname, is_new=bool(name) or mod=='#', solo_nb=solo_nb)
            mm.make(cells, all_cells, lib_path=lib_path)

Let's check we can import a test file:

In [ ]:
#| eval: false
shutil.rmtree('tmp', ignore_errors=True)
nb_export('../../tests/00_some.thing.ipynb', 'tmp')

g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a'])
test_eq(g['tmp'].some.thing.a, 1)

We'll also check that our 'everything' file exports correctly:

In [ ]:
#| eval: false
nb_export(everything_fn, 'tmp')

g = exec_new('import tmp.everything; from tmp.everything import *')
_alls = L("a b d e m n o p q".split())
for s in _alls.map("{}_y"): assert s in g, s
for s in "c_y_nall _f_y_nall g_n h_n i_n j_n k_n l_n".split(): assert s not in g, s
for s in _alls.map("{}_y") + ["c_y_nall", "_f_y_nall"]: assert hasattr(g['tmp'].everything,s), s

That notebook should also export one extra function to `tmp.some.thing`:

In [ ]:
#| eval: false
del(sys.modules['tmp.some.thing']) # remove from module cache
g = exec_new('import tmp.some.thing')
test_eq(g['tmp'].some.thing.__all__, ['a','h_n'])
test_eq(g['tmp'].some.thing.h_n(), None)

## Export -

In [ ]:
#| eval: false
Path('../nbdev/export.py').unlink(missing_ok=True)
nb_export('04_export.ipynb')

g = exec_new('import nbdev.export')
assert hasattr(g['nbdev'].export, 'nb_export')